In [36]:
import os
import streamlit as st
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from ragas.evaluation import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

In [37]:
# Carrega variáveis de ambiente
load_dotenv()
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

In [29]:
def format_docs(docs):
    """Formata os documentos recuperados."""
    return "\n\n".join(doc.page_content for doc in docs)

In [38]:
def main():
    st.title("Vehicle Manuals QA with RAGAS Evaluation")

    # Entrada do usuário
    question = st.text_input("Digite sua pergunta:")

    if question:
        # Inicialização de embeddings e modelo
        # Ajuste o nome do modelo de embeddings conforme necessário
        local_embeddings = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-large",
            model_kwargs={'trust_remote_code': True}
        )
        model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

        # Cria o cliente Qdrant
        try:
            qdrant_client = QdrantClient(
                url=QDRANT_URL,
                api_key=QDRANT_API_KEY,
            )

            # Instancia o Qdrant VectorStore já apontando para a coleção existente
            vectorstore = Qdrant(
                client=qdrant_client,
                collection_name="fluidos_chunkby-page_overlap0_multilingual-e5-large",
                embedding=local_embeddings
            )
        except Exception as e:
            st.error(f"Erro ao conectar ao Qdrant: {e}")
            return

        retriever = vectorstore.as_retriever()

        # Prompt do RAG
        RAG_TEMPLATE = """
        Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperado para responder à pergunta.
        Se você não souber a resposta, simplesmente diga que não sabe.
        <context>{context}</context> Responda à seguinte pergunta: {question}"""

        rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

        # Chain de execução do RAG
        rag_chain_from_docs = (
            RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
            | rag_prompt
            | model
            | RunnableLambda(lambda x: x.content if hasattr(x, 'content') else x)
        )

        rag_chain_with_source = RunnableParallel(
            {"context": retriever, "question": RunnablePassthrough()}
        ).assign(answer=rag_chain_from_docs)

        # Execução da cadeia RAG
        try:
            result = rag_chain_with_source.invoke(question)
        except Exception as e:
            st.error(f"Erro ao processar a cadeia RAG: {e}")
            return

        # Exibição da pergunta e resposta
        st.subheader("Pergunta:")
        st.write(result.get('question', '').strip())

        st.subheader("Resposta:")
        st.write(result.get('answer', 'Sem resposta disponível').strip())

        # Exibição dos documentos recuperados
        st.subheader("Documentos Recuperados:")
        for i, doc in enumerate(result.get('context', []), 1):
            st.markdown(f"**Documento {i}:**")
            st.markdown(f"**Fonte:** {doc.metadata.get('source', '').strip()}")
            st.markdown(f"**Conteúdo:** {doc.page_content[:500].strip()}...")

        # Preparação dos resultados de avaliação
        evaluation_result = {
            'query': result.get('question', ''),
            'result': result.get('answer', ''),
            'source_documents': result.get('context', []),
        }

        # Avaliação das métricas
        try:
            with st.spinner('Calculando métricas RAGAS...'):
                faithfulness_score = evaluate(faithfulness, evaluation_result)
                answer_relevancy_score = evaluate(answer_relevancy, evaluation_result)
                context_precision_score = evaluate(context_precision, evaluation_result)
                context_recall_score = evaluate(context_recall, evaluation_result)

            # Exibição dos resultados de avaliação
            st.subheader("Métricas de Avaliação RAGAS:")
            st.write(f"**Pontuação de Fidelidade:** {faithfulness_score:.2f}")
            st.write(f"**Pontuação de Relevância da Resposta:** {answer_relevancy_score:.2f}")
            st.write(f"**Pontuação de Precisão do Contexto:** {context_precision_score:.2f}")
            st.write(f"**Pontuação de Revocação do Contexto:** {context_recall_score:.2f}")

        except Exception as e:
            st.error(f"Erro ao calcular métricas RAGAS: {e}")

In [32]:
if __name__ == "__main__":
    main()

2024-12-18 10:03:34.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 10:03:34.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 10:03:34.738 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 10:03:34.738 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 10:03:34.738 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 10:03:34.738 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 10:03:34.739 Session state does not function when running a script without `streamlit run`
2024-12-18 10:03:34.739 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-18 10:03